<a href="https://colab.research.google.com/github/Vinicius-Jose/langchain_notebooks/blob/main/Llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* This notebook has the purpose to create an AI to decide the best formation and lineup to win a soccer match based on two national teams. Note: only works with national team who disputed the world cup 2022.

* This notebook use Llama, Langchain, Langgraph and integration with https://www.api-football.com/ to get information about statistics and squad

* Its necessary add an API_SPORT variable as a google colab secret, to use the API, you can get an key [here](https://www.api-football.com/)

# Llama Installation and Config

In [1]:
!pip install llama-stack langchain langgraph langchain-huggingface langchain-community langchain-ollama ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.7/292.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━

In [2]:
! export OLLAMA_HOST=127.0.0.1
! export OLLAMA_PORT=11434

In [3]:
!apt-get update
!apt-get install -y lshw
!apt-get install -y pciutils  # Install packages to use GPU

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,665 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,926 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://ppa.launchpadcontent.net/

In [4]:
# Install Llama
!curl -fsSL https://ollama.com/install.sh | sh
!curl -L https://ollama.com/download/ollama-linux-amd64.tgz -o ollama-linux-amd64.tgz
!sudo tar -C /usr -xzf ollama-linux-amd64.tgz

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1616M  100 1616M    0     0   170M      0  0:00:09  0:00:09 --:--:--  140M


In [16]:
# Initialize Llama server
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [26]:
# Download Llama model
!ollama pull llama3.1
!nohup ollama serve &

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 667b0c1932bc:   0% ▕▏ 851 KB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   1% ▕▏  29 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   2% ▕▏  92 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   3% ▕▏ 152 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   4% ▕▏ 173 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   4% ▕▏ 219 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   5% ▕▏ 266 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   6% ▕▏ 286 MB/4.9 GB                  pulling manifest 
pulling 

# Langgraph

In [189]:
from langgraph.graph import START, MessagesState, StateGraph

In [190]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "llama3.1",
    temperature = 0,
    num_predict = 300,
)

In [191]:
from langchain_core.tools import tool
import requests

try:
  from google.colab import userdata
  key = userdata.get('API_SPORT')
except:
  key = input("Enter your API_SPORT: ")


headers = {
'x-rapidapi-host': "v3.football.api-sports.io",
'x-apisports-key': key,
'x-rapidapi-key':key
}
url = "https://v3.football.api-sports.io/"

def get_team_id(national_team:str) -> int:
  url_formated = f"{url}/teams?name={national_team}"
  response = requests.get(url_formated, headers=headers, data={})
  response = response.json()
  return response["response"][0]["team"]["id"]

@tool
def  get_statistics_world_cup(national_team:str) -> dict:
    """ Get the statistics of a national team in the last world cup """
    team_id = get_team_id(national_team)
    url_formated = f"{url}/standings?season=2022&league=1&team={team_id}"
    response = requests.get(url_formated, headers=headers, data={})
    response = response.json()
    return response

@tool
def  get_squad(national_team:str) -> dict:
    """ Check the current squad of the national team"""
    team_id = get_team_id(national_team)
    url_formated = f"{url}/players/squads?team={team_id}"
    response = requests.get(url_formated, headers=headers, data={})
    return response.json()


@tool
def  get_player(player_id:str) -> dict:
    """ Return info about the player"""
    url_formated = f"{url}/players/profiles?player={player_id}"
    response = requests.get(url_formated, headers=headers, data={})
    return response.json()

In [192]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Literal, Annotated,Dict

class Router(BaseModel):
    country: str = Field(description="Name of the your national team")
    formation:Literal["4-4-2","4-3-3","3-5-2","4-2-3-1","5-3-2","3-4-3",
                      "4-1-4-1","4-5-1","3-6-1","4-4-1-1"] = Field(description="Formation choose to win the match against the natinal team")
    lineup_team:str = Field(description="Lineup you have choose for the next match with player names and position for the chooosen formation  on the current squad")
    lineup: Dict[str, str] = Field(description="Dictionary with the players positions in choosen formation as keys and player names as values from the lineup of your team.")
    explanation:str = Field(description="Detailed explanation  why this strategy will work and how to implement")



In [193]:
system_prompt = """
< Role >
You are an expert in soccer, and a coach assistant of {team} national team. You are a top-notch assistant coach who cares to make the best decision about national team tactics.
You have the ability to answer in the same language of the question
</ Role >

< Tools >
You have access to the following tools to help choose the best tactics against the national team above :

1. get_statistics_world_cup(national_team) - Get the statistics about the national team in the last World Cup, always pass the name of the national team in english
2. get_squad(national_team) - Get the list of current players from a national team, always pass the name of the national team in english, use only the players returned here to define the your lineup  for the tactics you choose, if you cant get the players return an empty list. Use this tool to determine the lineup of your team, use only players returned here.
</ Tools >

< Instructions >
You will need check the statiscts of our opponent and the squad of our opponent, than choose the best formation, and look at our squad to define the lineup, don't add players that is not present on our squad when you use .
Based on informations you get return the formation you choose of your national team  to win the other national team, the lineup, with the name of the players for the match, all players need to be of your team, and an explanation of why this strategy will work and how to implement.
Only use players returned from get_squad tool, inside response[0][players] when you pass the name of your team to the tool, do not mention players that is not on the list
</ Instructions >
"""

In [194]:
def get_messages(country:str) -> dict:
  user_prompt= f"What is the formation and lineup I need to win the next match against {country}?"
  return {"messages": [{
        "role": "user",
        "content": user_prompt
    }]}

In [195]:
team = "Brazil"
def create_prompt(state):
    return [
        {
            "role": "system",
            "content": system_prompt.format(
                team=team,

                )
        }
    ] + state['messages']

In [196]:
from langgraph.prebuilt import create_react_agent
llm_router = llm.with_structured_output(Router)
agent = create_react_agent(
    llm,
    tools=[get_statistics_world_cup,get_squad,get_player],
    prompt=create_prompt,
    response_format=Router
)

In [197]:
result = agent.invoke(get_messages("Italy"))

In [198]:
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

What is the formation and lineup I need to win the next match against Italy?
================================== Ai Message ==================================
Tool Calls:
  get_statistics_world_cup (b07b1a51-34b6-4441-8c94-f9dccc8bf780)
 Call ID: b07b1a51-34b6-4441-8c94-f9dccc8bf780
  Args:
    national_team: Italy
  get_squad (db3d7c5e-bdc5-49ca-b9ff-9aa557c25b81)
 Call ID: db3d7c5e-bdc5-49ca-b9ff-9aa557c25b81
  Args:
    national_team: Brazil
================================= Tool Message =================================
Name: get_statistics_world_cup

{"get": "standings", "parameters": {"season": "2022", "league": "1", "team": "768"}, "errors": [], "results": 0, "paging": {"current": 1, "total": 1}, "response": []}
================================= Tool Message =================================
Name: get_squad

{"get": "players/squads", "parameters": {"team": "6"}, "errors": [], "results": 1, "paging":

In [199]:
response = result["structured_response"]

In [200]:
response

Router(country='Brazil', formation='4-3-3', lineup_team='["Alisson Becker", "Alex Sandro", "Marquinhos", "Gabriel Magalhães", "Alex Sandro", "Bruno Guimarães", "Gerson", "Rodrygo", "Raphinha", "Endrick", "Vinícius Júnior"]', lineup={'1': 'Alisson Becker', '2': 'Alex Sandro', '3': 'Marquinhos', '4': 'Gabriel Magalhães', '5': 'Alex Sandro', '6': 'Bruno Guimarães', '7': 'Gerson', '8': 'Rodrygo', '9': 'Raphinha', '10': 'Endrick', '11': 'Vinícius Júnior'}, explanation='This lineup provides a good balance between defense and attack, with a solid defensive shape and creative players in midfield and attack.')

In [201]:
response.lineup

{'1': 'Alisson Becker',
 '2': 'Alex Sandro',
 '3': 'Marquinhos',
 '4': 'Gabriel Magalhães',
 '5': 'Alex Sandro',
 '6': 'Bruno Guimarães',
 '7': 'Gerson',
 '8': 'Rodrygo',
 '9': 'Raphinha',
 '10': 'Endrick',
 '11': 'Vinícius Júnior'}